In [126]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score



# Assuming the CSV files are in the same directory as your Python script or Jupyter Notebook

# Import Game data
games_data = pd.read_csv('nfl-big-data-bowl-2024/games.csv')

# Import Play data
plays_data = pd.read_csv('nfl-big-data-bowl-2024/plays.csv')

# Import Player data
players_data = pd.read_csv('nfl-big-data-bowl-2024/players.csv')

# Import Tackles data
tackles_data = pd.read_csv('nfl-big-data-bowl-2024/tackles.csv')

tracking_data_1 = pd.read_csv(f'nfl-big-data-bowl-2024/tracking_week_1.csv')

# Example for importing all weeks of Tracking data
# Adjust the range based on the actual weeks in your dataset
tracking_data_all_weeks = pd.concat([pd.read_csv(f'nfl-big-data-bowl-2024/tracking_week_{week}.csv') for week in range(1, 10)])

# Now you can use these DataFrames in your analysis

In [ ]:
# grouping similar positions

modified_players_data = players_data

In [10]:
tracking_data_1.head()

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
0,2022090800,56,35472.0,Rodger Saffold,1,2022-09-08 20:24:05.200000,76.0,BUF,left,88.37,27.27,1.62,1.15,0.16,231.74,147.90,NaN
1,2022090800,56,35472.0,Rodger Saffold,2,2022-09-08 20:24:05.299999,76.0,BUF,left,88.47,27.13,1.67,0.61,0.17,230.98,148.53,pass_arrived
2,2022090800,56,35472.0,Rodger Saffold,3,2022-09-08 20:24:05.400000,76.0,BUF,left,88.56,27.01,1.57,0.49,0.15,230.98,147.05,NaN
3,2022090800,56,35472.0,Rodger Saffold,4,2022-09-08 20:24:05.500000,76.0,BUF,left,88.64,26.90,1.44,0.89,0.14,232.38,145.42,NaN
4,2022090800,56,35472.0,Rodger Saffold,5,2022-09-08 20:24:05.599999,76.0,BUF,left,88.72,26.80,1.29,1.24,0.13,233.36,141.95,NaN


In [45]:
games_data.head()
players_data[players_data["position"] == 'WR']

,nflId,height,weight,birthDate,collegeName,position,displayName
7,33130,5-10,175,1986-12-01,California,WR,DeSean Jackson
26,37077,6-4,210,1988-07-31,Georgia,WR,A.J. Green
28,37079,6-3,220,1989-02-08,Alabama,WR,Julio Jones
37,37139,5-10,192,1990-08-22,Kentucky,WR,Randall Cobb
68,38696,6-2,198,1990-03-12,California,WR,Marvin Jones
...,...,...,...,...,...,...,...
1649,54938,6-0,185,NaN,Mississippi,WR,Braylon Sanders
1661,55071,5-10,180,NaN,Texas Tech,WR,Kaylon Geiger
1667,55133,6-0,180,NaN,Weber State,WR,Rashid Shaheed
1672,55158,6-2,205,NaN,Western Illinois,WR,Dennis Houston


In [28]:
#tom brady was tracked 2201 times in week 1
player_tracking = tracking_data_1[tracking_data_1['nflId'] == 29550]
player_tracking

,gameId,playId,nflId,displayName,frameId,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event


In [15]:
average_speed = player_tracking['s'].sum() / len(player_tracking['s'])
average_speed

1.6080508859609268

In [24]:
def calculate_average_speed(tracking_data, nflId):
    # Filter tracking data for the specific player
    player_tracking = tracking_data[tracking_data['nflId'] == nflId]
    average_speed = player_tracking['s'].sum() / len(player_tracking['s'])
    return average_speed

# Example usage:
# Assuming you have loaded the tracking data into a DataFrame named 'tracking_data'
# and the player data into a DataFrame named 'player_data'

# Replace 'your_nflId_here' with the actual nflId of the player you want to calculate the average speed for
nflId_to_calculate = 29550	  

average_speed = calculate_average_speed(tracking_data_all_weeks, nflId_to_calculate)
average_speed_min = average_speed * 60
print(f"Average speed for player with nflId {nflId_to_calculate}: {average_speed} yards/second")
print(f"Average speed for player with nflId {nflId_to_calculate}: {average_speed_min} yards/minute")

Average speed for player with nflId 29550: 2.399963427377221 yards/second
Average speed for player with nflId 29550: 143.99780564263324 yards/minute


In [59]:
def calculate_normalized_aggression_score(tackles_data, nflId):
    # Filter tackles data for the specific player
    player_tackles = tackles_data[tackles_data['nflId'] == nflId]

    if len(player_tackles) == 0:
        #print(f"Player with nflId {nflId} not found in the tackles data.") 
        return 0

    # Calculate normalized aggression score based on tackles, assisted tackles, forced fumbles, and missed tackles
    tackles = player_tackles['tackle'].sum()
    assisted_tackles = player_tackles['assist'].sum()
    forced_fumbles = player_tackles['forcedFumble'].sum()
    missed_tackles = player_tackles['pff_missedTackle'].sum()

    # Define a formula for the normalized aggression score (you can adjust the weights as needed)
    aggression_score = (tackles + assisted_tackles + forced_fumbles -
                        missed_tackles) / max(1, len(player_tackles))

    return aggression_score


# Example usage:
# Assuming you have loaded the tackles data into a DataFrame named 'tackles_data'

# Replace 'your_nflId_here' with the actual nflId of the player you want to calculate the normalized aggression score for
nflId_to_calculate_aggression_score = 29550

player_aggression_score = calculate_normalized_aggression_score(
    tackles_data, nflId_to_calculate_aggression_score)
if player_aggression_score is not None:
    print(
        f"Normalized Aggression Score for player with nflId {nflId_to_calculate_aggression_score}: {player_aggression_score:.2f}"
    )

Normalized Aggression Score for player with nflId 29550: 0.00


In [29]:
def calculate_bmi(player_data, nflId):
    # Filter player data for the specific player
    player_info = player_data[player_data['nflId'] == nflId]

    if len(player_info) == 0:
        print(f"Player with nflId {nflId} not found.")
        return None

    # Extract weight and height
    weight_kg = player_info['weight'].values[0] * 0.453592  # Convert pounds to kilograms
    height_meters = convert_height_to_meters(player_info['height'].values[0])  # Convert feet and inches to meters

    # Calculate BMI
    bmi = weight_kg / (height_meters ** 2)

    return bmi

def convert_height_to_meters(height):
    # Convert height from feet and inches to meters
    feet, inches = map(int, height.split('-'))
    height_meters = (feet * 12 + inches) * 0.0254  # 1 foot = 12 inches, 1 inch = 0.0254 meters
    return height_meters

# Example usage:
# Assuming you have loaded the player data into a DataFrame named 'player_data'

# Replace 'your_nflId_here' with the actual nflId of the player you want to calculate the BMI for
nflId_to_calculate_bmi = 29550  

player_bmi = calculate_bmi(players_data, nflId_to_calculate_bmi)
if player_bmi is not None:
    print(f"BMI for player with nflId {nflId_to_calculate_bmi}: {player_bmi:.2f}")

BMI for player with nflId 29550: 39.92


In [38]:
#no interceptions were recorded

player_interceptions = plays_data[plays_data['passResult'] == 'IN']
player_interceptions

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2


In [44]:
#no interceptions were recorded

player_catches = plays_data[plays_data['passResult'] == 'C']
player_catches

,gameId,playId,ballCarrierId,ballCarrierDisplayName,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,...,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPoints,expectedPointsAdded,foulName1,foulName2,foulNFLId1,foulNFLId2
0,2022100908,3537,48723,Parker Hesse,(7:52) (Shotgun) M.Mariota pass short middle t...,4,1,10,ATL,TB,...,0.976785,0.023215,-0.006110,0.006110,2.360609,0.981955,NaN,NaN,NaN,NaN
2,2022091111,1148,42547,Darren Waller,(8:57) D.Carr pass short middle to D.Waller to...,2,2,5,LV,LAC,...,0.756661,0.243339,-0.037409,0.037409,1.312855,1.133666,NaN,NaN,NaN,NaN
5,2022103001,2165,54616,Tyler Allgeier,(10:14) (Shotgun) M.Mariota pass short left to...,3,3,17,ATL,CAR,...,0.599479,0.400521,-0.030628,0.030628,-1.484904,-0.639661,NaN,NaN,NaN,NaN
9,2022092511,2903,46096,Rashaad Penny,(:35) (Shotgun) G.Smith pass short left to R.P...,3,1,10,SEA,ATL,...,0.344336,0.655664,-0.018103,0.018103,0.986438,-0.477845,NaN,NaN,NaN,NaN
12,2022091102,3336,46331,Ross Dwelley,(11:55) (Shotgun) T.Lance pass short middle to...,4,3,10,SF,CHI,...,0.632420,0.367580,-0.085709,0.085709,-0.208933,1.777092,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12472,2022092500,2194,41292,Jarvis Landry,(13:41) J.Winston pass short left to J.Landry ...,3,1,10,NO,CAR,...,0.831099,0.168901,-0.021496,0.021496,1.399707,0.840568,NaN,NaN,NaN,NaN
12478,2022100213,811,40011,Travis Kelce,(1:27) P.Mahomes pass short left to T.Kelce to...,1,2,2,KC,TB,...,0.167661,0.832339,0.001442,-0.001442,1.018668,0.354830,NaN,NaN,NaN,NaN
12479,2022092504,2509,52725,Ben Ellefson,(8:31) K.Cousins pass short right to B.Ellefso...,3,1,10,MIN,DET,...,0.569608,0.430392,0.024005,-0.024005,1.105009,0.715667,NaN,NaN,NaN,NaN
12480,2022100910,105,44959,George Kittle,(13:43) (Shotgun) J.Garoppolo pass deep left t...,1,3,6,SF,CAR,...,0.395102,0.604898,-0.068616,0.068616,0.172976,2.691196,NaN,NaN,NaN,NaN


In [103]:
player_position_index = players_data.index[players_data["nflId"] == 46096]
player_position_index.shape

(1,)

In [105]:
players_data["nflId"].shape

(1683,)

In [114]:
df = pd.DataFrame([[0, 2, 3], [0, 4, 1], [10, 20, 30]],index=[4, 5, 6], columns=['A', 'B', 'C'])
df.at[4, 'B']

2

In [117]:
player_position_index = players_data.index[players_data["nflId"] == 46096]
player_position = players_data.at[player_position_index.values[0], 'position']
player_position

'RB'

In [118]:
feature_table = pd.DataFrame()
nflIds = players_data["nflId"]

for id in nflIds:
    player_bmi = calculate_bmi(players_data, id)
    player_aggression_score = calculate_normalized_aggression_score(tackles_data, id)
    average_speed = calculate_average_speed(tracking_data_all_weeks, id)
    passes_caught = count_passes_caught(plays_data, id)
    player_position_index = players_data.index[players_data["nflId"] == id]
    player_position = players_data.at[player_position_index.values[0], 'position']
    new_data = pd.DataFrame(data={'nflId': [id],'BMI': [player_bmi],'Average Speed': [player_aggression_score],'Aggression Score': [player_aggression_score],'Passes Caught': [passes_caught],'Position': [player_position]})
    # Concatenate the existing data with the new table
    feature_table = pd.concat([feature_table, new_data], ignore_index=True)

# Display the updated table
feature_table 


,nflId,BMI,Average Speed,Aggression Score,Passes Caught,Position
0,25511,27.387557,0.0,0.0,0,QB
1,29550,39.924971,0.0,0.0,0,T
2,29851,28.887970,0.0,0.0,0,QB
3,30842,30.854606,0.0,0.0,1,TE
4,33084,26.413777,0.0,0.0,0,QB
...,...,...,...,...,...,...
1678,55200,30.739046,1.0,1.0,0,DT
1679,55212,31.193262,0.0,0.0,0,ILB
1680,55239,38.517294,1.0,1.0,0,DT
1681,55240,24.407537,1.0,1.0,0,CB


In [137]:

# Define features (X) and target variable (y)
features = feature_table[['BMI', 'Average Speed', 'Aggression Score', 'Passes Caught']] 
target = feature_table['Position'] 

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

# Initialize the decision tree classifier
classifier = DecisionTreeClassifier(max_depth=5, criterion='entropy', random_state=42)

# Train the model
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Display Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Display Precision Score
print("Precision Score:")
print(precision_score(y_test, y_pred, average='macro'))

# Display Recall Score
print("Recall Score:")
print(recall_score(y_test, y_pred,average='macro'))

# Display Feature Importances
feature_importance = pd.DataFrame(classifier.feature_importances_ , index=features.columns).sort_values
print(feature_importance)

Accuracy: 0.49
Classification Report:
              precision    recall  f1-score   support

           C       0.00      0.00      0.00        16
          CB       0.54      0.81      0.64        36
          DB       0.00      0.00      0.00         1
          DE       0.70      0.50      0.58        28
          DT       0.68      0.84      0.75        25
          FB       0.00      0.00      0.00         3
          FS       0.17      0.07      0.10        15
           G       0.40      0.27      0.32        15
         ILB       0.50      0.10      0.17        20
         MLB       0.00      0.00      0.00         1
          NT       0.00      0.00      0.00         7
         OLB       0.36      0.59      0.45        22
          QB       0.55      0.43      0.48        14
          RB       0.00      0.00      0.00        23
          SS       0.00      0.00      0.00        14
           T       0.42      0.90      0.58        21
          TE       0.49      0.50      0.49

/Users/smithsonarrey/miniconda3/envs/nfl_player_classifier/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/smithsonarrey/miniconda3/envs/nfl_player_classifier/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/smithsonarrey/miniconda3/envs/nfl_player_classifier/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter t

# Below this line is all Chat GPT Starter Code

In [67]:
def count_passes_caught(play_data, ball_carrier_id, pass_result='C'):
    # Filter play data for completed passes where the specified player is the ball carrier
    player_passes_caught = play_data[(play_data['ballCarrierId'] == ball_carrier_id) & (play_data['passResult'] == pass_result)]

    # Count the number of passes caught
    num_passes_caught = len(player_passes_caught)

    return num_passes_caught

# Example usage:
# Assuming you have loaded the play data into a DataFrame named 'plays_data'

# Replace 'your_ballCarrierId_here' with the actual ballCarrierId of the player you want to analyze
ballCarrierId_to_analyze = 33130	  

passes_caught = count_passes_caught(plays_data, ballCarrierId_to_analyze)
print(f"Number of passes caught by player with ballCarrierId {ballCarrierId_to_analyze}: {passes_caught}")


Number of passes caught by player with ballCarrierId 33130: 2


In [49]:
def calculate_average_time_per_game(play_data, tracking_data, ball_carrier_id):
    # Filter play data for plays where the specified player is the ball carrier
    player_plays = play_data[play_data['ballCarrierId'] == ball_carrier_id]

    if len(player_plays) == 0:
        print(f"Player with nflId {ball_carrier_id} not found in the play data.")
        return None

    # Merge play data with tracking data to get the corresponding gameId and playId
    merged_data = pd.merge(player_plays[['gameId', 'playId']], tracking_data, on=['gameId', 'playId'], how='inner')

    # Calculate the average time the player carries the ball per game
    average_time_per_game = merged_data.groupby('gameId')['time'].max().mean()

    return average_time_per_game

# Example usage:
# Assuming you have loaded the play data into a DataFrame named 'plays_data'
# and the tracking data into a DataFrame named 'tracking_data'

# Replace 'your_nflId_here' with the actual nflId of the player you want to analyze
nflId_to_analyze_average_time = 33130

average_time_per_game = calculate_average_time_per_game(plays_data, tracking_data_all_weeks, nflId_to_analyze_average_time)
if average_time_per_game is not None:
    print(f"Average time per game for player with nflId {nflId_to_analyze_average_time}: {average_time_per_game:.2f} seconds")


TypeError: Could not convert string '2022-11-07 21:08:32.799999' to numeric

In [42]:
def count_passes_thrown(play_data, passer_id):
    # Filter play data for passes thrown where the specified player is the passer
    player_passes_thrown = play_data[(play_data['passResult'] == 'C') & (play_data['passer_player_id'] == passer_id)]

    # Count the number of passes thrown
    num_passes_thrown = len(player_passes_thrown)

    return num_passes_thrown

# Example usage:
# Assuming you have loaded the play data into a DataFrame named 'plays_data'

# Replace 'your_passerId_here' with the actual passer_player_id of the player you want to analyze
passerId_to_analyze_passes_thrown = 25511	  

passes_thrown_count = count_passes_thrown(plays_data, passerId_to_analyze_passes_thrown)
print(f"Number of passes thrown by player with passer_player_id {passerId_to_analyze_passes_thrown}: {passes_thrown_count}")


KeyError: 'passer_player_id'